# 1. 모델 평가지표

PPT 참고

In [1]:
y_pred = [[0.99, 0, 0.01, 0, 0],
[0.2, 0.1, 0.3, 0.2 , 0.2],
[0.1, 0.6, 0.1, 0.1, 0.1],
[0.5, 0.2, 0.1, 0.1, 0.1]]

In [2]:
y_true = [[1,0,0,0,0],
[0,0,1,0,0],
[0,1,0,0,0],
[0,0,0,0,1]]

In [41]:
from sklearn.metrics import log_loss

In [4]:
log_loss(y_true, y_pred)

1.0068584642348692

In [5]:
from math import log

In [6]:
import numpy as np

In [7]:
-(log(0.99) + log(0.3) + log(0.6) + log(0.1)) / 4

1.0068584642348684

# 2. 검증셋 구축

CV 스코어랑 LB 스코어가 어떻게 되시나요?

- cross validation score
- leaderboard score
- A변화를 줬을 때, CV가 올라갔다, LB가 올라갔다 (이상적인) -> CV내려갈 때, LB도 내려가는 경우
- A변화를 줬을 때, CV가 올라갈 때 LB가 내려가는 혹은 CV가 내려갈 때 LB가 올라가는 -> (최악)

In [7]:
import os
import pandas as pd 
import numpy as np
import multiprocessing # 여러 개의 일꾼 (cpu)들에게 작업을 분산시키는 역할
from multiprocessing import Pool 
from functools import partial # 함수가 받는 인자들 중 몇개를 고정 시켜서 새롭게 파생된 함수를 형성하는 역할
from data_loader_v2 import data_loader_v2 # 자체적으로 만든 data loader version 2.0 ([데이콘 15회 대회] 데이터 설명 및 데이터 불러오기 영상 참조)
from tqdm import tqdm

In [2]:
train_folder = '../train/'
test_folder = '../test/'
train_label_path = '../etc/train_label.csv'

train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

In [3]:
# 모든 csv 파일의 상태_B로 변화는 시점이 같다라고 가정
# 하지만, 개별 csv파일의 상태_B로 변화는 시점은 상이할 수 있음
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(tqdm(pool.imap(func_fixed, files), total = len(files)))
        pool.close()
        pool.join()       
    combined_df = pd.concat(df_list)    
    return combined_df

## 2.1 데이터 불러오기

In [4]:
train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, event_time=10, nrows=40)

100%|████████████████████████████████████████████████████████████████████████████████| 827/827 [01:57<00:00,  7.03it/s]


In [5]:
test = data_loader_all_v2(data_loader_v2, test_list, folder=test_folder, train_label=None, event_time=10, nrows=40)

100%|████████████████████████████████████████████████████████████████████████████████| 718/718 [01:40<00:00,  7.15it/s]


In [1]:
import time
import pandas as pd

In [2]:
start = time.time()
train = pd.read_csv('../train.csv', index_col = 0)
test = pd.read_csv('../test.csv', index_col = 0)
print(time.time() - start)

105.0934042930603


## 2.2 데이터 분리 (train, val)

In [43]:
train['label'].value_counts() / train['label'].value_counts().sum()

110    0.029021
17     0.025393
118    0.024184
114    0.024184
113    0.022975
         ...   
137    0.001209
153    0.001209
185    0.001209
42     0.001209
135    0.001209
Name: label, Length: 198, dtype: float64

### 2.2.1 랜덤하게 train test split

In [4]:
from sklearn.model_selection import train_test_split #한번만 분리

In [5]:
X_train = train.drop(['label'], axis=1)
y_train = train['label']

In [63]:
X_trn, X_val, y_trn, y_val = train_test_split(X_train, y_train, test_size = 0.2, shuffle = True, random_state = 0)
#test_size = 0.2 비율로(20%)->여기서는 X_val이랑 y_val이 test의 예측값이랑 label값, 순서에 영향이 없으니까 shuffle을 해줌

In [7]:
X_trn.shape, X_val.shape, y_trn.shape, y_val.shape

((19848, 5121), (4962, 5121), (19848,), (4962,))

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
def train_model(X, y):
    
    model = RandomForestClassifier(random_state=0, verbose=1, n_jobs=-1)
    model.fit(X, y)
    
    return model

In [17]:
model = train_model(X_trn, y_trn)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.6min finished


In [59]:
y_trn.value_counts()[[137, 153, 42]]

137    26
153    25
42     21
Name: label, dtype: int64

다행히도 198개의 라벨이 모두 있습니다.

In [29]:
y_pred = model.predict_proba(X_val)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished


In [33]:
y_val.values 
# 현재 data형태가 y_pred랑 다름 따라서 67, 68, 83...번째 값에 1이 들어가게 형태 변환

array([ 67,  68,  83, ...,  28,  68, 193], dtype=int64)

In [39]:
from sklearn.preprocessing import label_binarize 
y_val_binarize = label_binarize(y_val, classes = range(198))
# 현재 data형태가 y_pred랑 다름 따라서 67, 68, 83...번째 값에 1이 들어가게 형태 변환 -> 이 일을 label_binarize 이 친구가 해줌.

In [42]:
log_loss(y_val_binarize, y_pred)

0.8721437176576072

### 2.2.2 Stratified K fold

In [45]:
from sklearn.model_selection import StratifiedKFold 
skf = StratifiedKFold(n_splits=5) #skf라는 객체 생성
# y_train에서 데이터 개수가 다르기 때문에 특히 적은 개수의 데이터에서 분리 할때 훈련할 데이터랑 label데이터랑 적합하게 분리 해야함.
# y_train에서 y_val 비율을 똑같이 줘서 split하기위해 StratifiedKFold을 도입.

5

In [49]:
for trn_idx, val_idx in skf.split(X_train, y_train):
    y_trn, y_val = y_train.iloc[trn_idx], y_train.iloc[val_idx]
    break

In [ ]:
for trn_idx, val_idx in skf.split(X_train, y_train):
    X_trn, X_val ,y_trn, y_val = X_train.iloc[trn_idx], X_train.iloc[val_idx] ,y_train.iloc[trn_idx], y_train.iloc[val_idx]
    trained_model = train_model(X_trn, y_trn)
    y_pred = trained_model.predict_proba(X_val)
    log_loss(y_val_binarize, y_pred)

In [51]:
y_trn.value_counts()

110    576
17     504
118    480
114    480
113    456
      ... 
137     24
153     24
185     24
42      24
135     24
Name: label, Length: 198, dtype: int64

In [53]:
y_val.value_counts()

110    144
17     126
118    120
114    120
117    114
      ... 
45       6
49       6
98       6
57       6
195      6
Name: label, Length: 198, dtype: int64

In [60]:
import lightgbm as lgb

In [82]:
model_lgb = lgb.LGBMClassifier(n_estimators=100)

In [94]:
model_lgb.fit(X_trn.iloc[:,:1000], y_trn, verbose = 10)

LGBMClassifier()

In [95]:
lgb_pred = model_lgb.predict_proba(X_val.iloc[:,:1000])